In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path("..").absolute()))

from flattened_inference_debug import flattened_run_debug


In [ ]:
# Setup parameters
event = Path("test_event.npz")
bank_folder = Path("test_bank")
n_int = 2**12
n_ext = 512
n_phi = 64
n_t = 64
blocksize = 1024
single_detector_blocksize = 1024
i_int_start = 0
seed = 42
load_inds = False
inds_path = None
size_limit = 10**6
draw_subset = False
n_draws = None
event_dir = Path("test_event")
rundir = None
coherent_score_min_n_effective_prior = 100
max_incoherent_lnlike_drop = 1000
max_bestfit_lnlike_diff = 20
mchirp_guess = None
extrinsic_samples = None
n_phi_incoherent = 32
preselected_indices = None
coherent_posterior_kwargs = {}


In [ ]:
# Run the flattened inference
results = flattened_run_debug(
    event=event,
    bank_folder=bank_folder,
    n_int=n_int,
    n_ext=n_ext,
    n_phi=n_phi,
    n_t=n_t,
    blocksize=blocksize,
    single_detector_blocksize=single_detector_blocksize,
    i_int_start=i_int_start,
    seed=seed,
    load_inds=load_inds,
    inds_path=inds_path,
    size_limit=size_limit,
    draw_subset=draw_subset,
    n_draws=n_draws,
    event_dir=event_dir,
    rundir=rundir,
    coherent_score_min_n_effective_prior=coherent_score_min_n_effective_prior,
    max_incoherent_lnlike_drop=max_incoherent_lnlike_drop,
    max_bestfit_lnlike_diff=max_bestfit_lnlike_diff,
    mchirp_guess=mchirp_guess,
    extrinsic_samples=extrinsic_samples,
    n_phi_incoherent=n_phi_incoherent,
    preselected_indices=preselected_indices,
    coherent_posterior_kwargs=coherent_posterior_kwargs,
)


In [ ]:
# All intermediate variables are now accessible in results
print(f"Number of selected samples: {len(results['inds'])}")
print(f"\nFirst detector: {results['det_name']}")
print(f"Sky position (lat, lon): ({results['lat']:.6f}, {results['lon']:.6f})")

print(f"\nOriginal par_dic_0 keys: {results['par_dic_0'].keys()}")
print(f"Original t_geocenter: {results['par_dic_0']['t_geocenter']:.6f}")
print(f"Original psi: {results['par_dic_0']['psi']:.6f}")
print(f"Original d_luminosity: {results['par_dic_0']['d_luminosity']:.6f}")

print(f"\nTransformed par_dic keys: {results['par_dic_transformed'].keys()}")
print(f"Transformed t_geocenter: {results['par_dic_transformed']['t_geocenter']:.6f}")
print(f"Transformed psi: {results['par_dic_transformed']['psi']:.6f}")
print(f"Transformed d_luminosity: {results['par_dic_transformed']['d_luminosity']:.6f}")

print(f"\nTime grid shape: {results['t_grid_single'].shape}")
print(f"Timeshifts shape: {results['timeshifts_dbt_single'].shape}")
print(f"r_iotp shape: {results['r_iotp'].shape}")
print(f"lnlike_iot shape: {results['lnlike_iot_single'].shape}")

print(f"\nFirst 10 lnlike_i values: {results['lnlike_i_single'][:10]}")
print(f"First 10 incoherent_lnlikes: {results['incoherent_lnlikes'][:10]}")


In [ ]:
# Now you can investigate the r_iotp values and compare to parameters
import numpy as np
import matplotlib.pyplot as plt

# Example: look at the first intrinsic sample
i_idx = 0  # First intrinsic sample in the batch

# Get all the orbital phase and time indices where lnlike was computed
r_sample = results['r_iotp'][i_idx]  # shape: (n_phi, n_t, 2)
lnlike_sample = results['lnlike_iot_single'][i_idx]  # shape: (n_phi, n_t)

# Find the maximum likelihood point
max_idx = np.unravel_index(lnlike_sample.argmax(), lnlike_sample.shape)
o_max, t_max = max_idx

print(f"Maximum likelihood found at:")
print(f"  Orbital phase index (o): {o_max} / {lnlike_sample.shape[0]}")
print(f"  Time index (t): {t_max} / {lnlike_sample.shape[1]}")
print(f"  lnlike value: {lnlike_sample[o_max, t_max]:.6f}")
print(f"\nCorresponding r_iotp value: {r_sample[o_max, t_max]}")

# Now you can reverse-engineer the parameters from this r_iotp


In [ ]:
# Verify reconstruction: convert r_iotp back to parameters and check against coherent likelihood
from pathlib import Path
import pandas as pd
from cogwheel.prior import read_json
from cogwheel.gw_utils import get_geocenter_delays

# Load bank config and intrinsic bank once
bank_config_dict = read_json(bank_folder / "bank_config.json")
intrinsic_bank_df = pd.read_feather(bank_folder / "intrinsic_sample_bank.feather")

# Number of samples to compare
n_comparisons = 1

# Compare for multiple samples
for i_idx in range(n_comparisons):
    print(f"\n{'='*80}")
    print(f"Comparison for sample {i_idx + 1}/{n_comparisons}")
    print(f"{'='*80}")
    
    # Get the max likelihood point
    max_idx = np.unravel_index(
        results["lnlike_iot_single"][i_idx].argmax(),
        results["lnlike_iot_single"][i_idx].shape,
    )
    o_max, t_max = max_idx
    r_max = results["r_iotp"][i_idx, o_max, t_max]

    # Reconstruct psi and d_luminosity from r
    psi_recon, d_luminosity_recon = results["sdp"].bestfit_response_to_psi_and_d_luminosity(
        r_max, results["det_name"]
    )

    # Reconstruct other parameters
    phi_ref_recon = o_max / n_phi_incoherent * 2 * np.pi
    t_geocenter_recon = (
        results["par_dic_transformed"]["t_geocenter"]
        + results["t_grid_single"][t_max]
        - results["sdp"].likelihood.event_data.tcoarse
        - get_geocenter_delays(results["det_name"], results["lat"], results["lon"])[0]
    )

    # Get intrinsic parameters from bank
    intrinsic_params = intrinsic_bank_df.iloc[results["batch_intrinsic_indices"][i_idx]].to_dict()
    intrinsic_params["f_ref"] = bank_config_dict["f_ref"]
    intrinsic_params["l1"] = 0
    intrinsic_params["l2"] = 0

    # Combine all parameters
    reconstructed_par_dic = intrinsic_params | {
        "ra": results["par_dic_transformed"]["ra"],
        "dec": results["par_dic_transformed"]["dec"],
        "t_geocenter": t_geocenter_recon,
        "phi_ref": phi_ref_recon,
        "psi": psi_recon,
        "d_luminosity": d_luminosity_recon,
    }

    print("Reconstructed parameters:")
    print(
        f"  psi: {psi_recon:.6f} (transformed had: {results['par_dic_transformed']['psi']:.6f})"
    )
    print(
        f"  d_luminosity: {d_luminosity_recon:.6f} (transformed had: {results['par_dic_transformed']['d_luminosity']:.6f})"
    )
    print(f"  phi_ref: {phi_ref_recon:.6f}")
    print(f"  t_geocenter: {t_geocenter_recon:.6f}")

    # Now compute likelihood using these reconstructed parameters
    lnlike_from_params = results["coherent_posterior"].likelihood.lnlike(
        reconstructed_par_dic
    )
    lnlike_from_barebones = results["lnlike_iot_single"][i_idx, o_max, t_max]

    print(f"\nLikelihood comparison:")
    print(f"  From reconstructed params: {lnlike_from_params:.6f}")
    print(f"  From barebones calculation: {lnlike_from_barebones:.6f}")
    print(f"  Difference: {lnlike_from_params - lnlike_from_barebones:.6f}")

    if abs(lnlike_from_params - lnlike_from_barebones) < 0.01:
        print("✓ Likelihoods match! Traceability verified.")
    else:
        print("✗ Likelihoods don't match. Need to investigate further.")

print(f"\n{'='*80}")
print(f"Completed {n_comparisons} comparison(s)")
print(f"{'='*80}")
